In [167]:
# Import Library
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

data = pd.read_csv('datasets-jadi.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 18 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   NO                                      561 non-null    int64  
 1   NAMA                                    561 non-null    object 
 2   USIA                                    561 non-null    float64
 3   PARITAS                                 561 non-null    int64  
 4   JARAK KELAHIRAN                         561 non-null    int64  
 5   PE/Non PE                               561 non-null    int64  
 6   RIW HIPERTENSI_Tidak                    561 non-null    float64
 7   RIW HIPERTENSI_Ya                       561 non-null    float64
 8   RIW PE_Ada                              561 non-null    float64
 9   RIW PE_Tidak                            561 non-null    float64
 10  OBESITAS_Tidak                          561 non-null    float6

In [168]:
#Persiapan Modelling

data = data.drop(['NAMA', 'NO'], axis=1) #dihapus karena tidak diperlukan untuk modeling

# Pisahkan variabel target (Preeklamsi)
X = data.drop('PE/Non PE', axis=1) 
y = data['PE/Non PE']

# Train-Test Split 75% 25%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [169]:
# Membangun Model Naive Bayes
model_nb = GaussianNB()
model_nb.fit(X_train, y_train)

# Melakukan Prediksi
y_pred_nb = model_nb.predict(X_test)

# Akurasi
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print("Akurasi Naive Bayes:", accuracy_nb)
percent_accuracy = "{:.2f}%".format(accuracy_nb * 100)
print(f"Akurasi: {percent_accuracy}")

# Confusion Matrix
cm_nb = confusion_matrix(y_test, y_pred_nb)
print("\nConfusion Matrix Naive Bayes:\n", cm_nb)

Akurasi Naive Bayes: 0.07801418439716312
Akurasi: 7.80%

Confusion Matrix Naive Bayes:
 [[  1 127]
 [  3  10]]


In [170]:
# Membangun Model KNN
model_knn = KNeighborsClassifier(n_neighbors=5)  
model_knn.fit(X_train, y_train)

y_pred_knn = model_knn.predict(X_test)

# Akurasi
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("Akurasi KNN:", accuracy_knn)
percent_accuracy = "{:.2f}%".format(accuracy_knn * 100)
print(f"Akurasi: {percent_accuracy}")

# Confusion Matrix
cm_knn = confusion_matrix(y_test, y_pred_knn)
print("\nConfusion Matrix KNN:\n", cm_knn)

Akurasi KNN: 0.9148936170212766
Akurasi: 91.49%

Confusion Matrix KNN:
 [[127   1]
 [ 11   2]]


In [171]:
# Membangun Model Decision Tree
model_dt = DecisionTreeClassifier()
model_dt.fit(X_train, y_train)

y_pred_dt = model_dt.predict(X_test)

# Akurasi
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Akurasi Decision Tree:", accuracy_dt)
percent_accuracy = "{:.2f}%".format(accuracy_knn * 100)
print(f"Akurasi: {percent_accuracy}")

# Confusion Matrix
cm_dt = confusion_matrix(y_test, y_pred_dt)
print("\nConfusion Matrix Decision Tree:\n", cm_dt)

Akurasi Decision Tree: 0.9078014184397163
Akurasi: 91.49%

Confusion Matrix Decision Tree:
 [[122   6]
 [  7   6]]


In [172]:
from sklearn.feature_selection import SelectKBest, chi2

# Menerapkan SelectKBest
k_best = 5 
bestfeatures = SelectKBest(score_func=chi2, k=k_best)
fit = bestfeatures.fit(X, y)

# Membuat DataFrame untuk score dan nama kolom
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

# Menggabungkan DataFrame
featureScores = pd.concat([dfcolumns, dfscores], axis=1)
featureScores.columns = ['Fitur', 'Skor'] 

# Menampilkan 5 fitur terbaik
print(featureScores.nlargest(k_best, 'Skor'))

                                   Fitur        Skor
5                             RIW PE_Ada  193.911607
4                      RIW HIPERTENSI_Ya  124.314050
11  RIW HIPERTENSI/PE DALAM KELUARGA_Ada   36.232411
8                            OBESITAS_Ya   20.492345
3                   RIW HIPERTENSI_Tidak   12.969773


In [173]:
# Buat DataFrame baru dengan fitur yang dipilih
X_selected = X[featureScores.nlargest(k_best, 'Skor')['Fitur']]  

# Train-Test Split dengan data baru (split data tetap sama)
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.25, random_state=42
)

In [174]:
#Naive Bayes setelah feature selection
model_nb = GaussianNB()
model_nb.fit(X_train, y_train)
y_pred_nb = model_nb.predict(X_test)

accuracy_nb = accuracy_score(y_test, y_pred_nb)
cm_nb = confusion_matrix(y_test, y_pred_nb)

print("\nNaive Bayes - Akurasi:", accuracy_nb)
percent_accuracy = "{:.2f}%".format(accuracy_nb * 100)
print(f"Akurasi: {percent_accuracy}")

print("Confusion Matrix:\n", cm_nb)


Naive Bayes - Akurasi: 0.8936170212765957
Akurasi: 89.36%
Confusion Matrix:
 [[116  12]
 [  3  10]]


In [175]:
# KNN setelah feature selection
model_knn = KNeighborsClassifier(n_neighbors=5)  
model_knn.fit(X_train, y_train)
y_pred_knn = model_knn.predict(X_test)

accuracy_knn = accuracy_score(y_test, y_pred_knn)
cm_knn = confusion_matrix(y_test, y_pred_knn)

print("\nKNN - Akurasi:", accuracy_knn)
percent_accuracy = "{:.2f}%".format(accuracy_knn * 100)
print(f"Akurasi: {percent_accuracy}")

print("Confusion Matrix:\n", cm_knn)


KNN - Akurasi: 0.9219858156028369
Akurasi: 92.20%
Confusion Matrix:
 [[125   3]
 [  8   5]]


In [176]:
#Decision Tree setelah feature selection
model_dt = DecisionTreeClassifier()
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)

accuracy_dt = accuracy_score(y_test, y_pred_dt)
cm_dt = confusion_matrix(y_test, y_pred_dt)

print("\nDecision Tree - Akurasi:", accuracy_dt)
percent_accuracy = "{:.2f}%".format(accuracy_dt * 100)
print(f"Akurasi: {percent_accuracy}")


print("Confusion Matrix:\n", cm_dt)


Decision Tree - Akurasi: 0.9290780141843972
Akurasi: 92.91%
Confusion Matrix:
 [[125   3]
 [  7   6]]
